In [6]:
import requests
from bs4 import BeautifulSoup
from abc import ABC, abstractmethod
from typing import List, Dict
import json
import os

# Định nghĩa lớp CarInfo (đơn giản hóa)
class CarInfo:
    def __init__(self, brand: str, url: str):
        self.brand = brand
        self.url = url

    def __dict__(self):
        return {"brand": self.brand, "url": self.url}

# Lớp cơ sở BaseScraper
class BaseScraper(ABC):
    def __init__(self, base_url: str):
        self.base_url = base_url
        self.session = requests.Session()
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        }
    
    def get_soup(self, url: str) -> BeautifulSoup:
        response = self.session.get(url, headers=self.headers)
        return BeautifulSoup(response.content, 'html.parser')

    @abstractmethod
    def get_all_car_urls(self) -> List[Dict[str, str]]:
        pass

# Lớp Website2Scraper để crawl
class Website2Scraper(BaseScraper):
    def save_to_json(self, filename: str, data: List[Dict]):
        if os.path.exists(filename):
            with open(filename, 'r', encoding='utf-8') as f:
                try:
                    existing_data = json.load(f)
                except json.JSONDecodeError:
                    existing_data = []
        else:
            existing_data = []
        
        existing_data.extend(data)
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(existing_data, f, ensure_ascii=False, indent=2)

    def get_all_car_urls(self) -> List[Dict[str, str]]:
        all_car_urls = []
        try:
            # Lấy HTML từ base_url
            soup = self.get_soup(self.base_url)
            print("Đã truy cập trang web thành công.")

            # Tìm danh sách thương hiệu xe (cập nhật bộ chọn dựa trên cấu trúc mới)
            # Giả sử danh sách thương hiệu nằm trong thẻ ul với class "brand-list"
            brand_section = soup.find("ul", class_="sds-list")  # Cập nhật class nếu cần
            if not brand_section:
                print("Không tìm thấy danh sách thương hiệu.")
                return all_car_urls

            # Lấy tất cả các liên kết thương hiệu
            brand_links = brand_section.find_all("a")
            for link in brand_links:
                brand_name = link.text.strip()
                brand_href = link.get("href")
                if brand_href:
                    full_url = brand_href if brand_href.startswith("http") else f"{self.base_url.rstrip('/')}/{brand_href.lstrip('/')}"
                    all_car_urls.append({"brand": brand_name, "url": full_url})
                    print(f"Đã tìm thấy: {brand_name} - {full_url}")

            # Lưu kết quả vào file JSON
            if all_car_urls:
                self.save_to_json("dataset/car_brands.json", all_car_urls)
                print("Đã lưu danh sách thương hiệu vào 'dataset/car_brands.json'.")

        except Exception as e:
            print(f"Lỗi khi crawl: {e}")

        return all_car_urls

# Chạy thử
if __name__ == "__main__":
    # Khởi tạo scraper với URL mới
    scraper = Website2Scraper("https://www.cars.com/new-cars/")
    car_brands = scraper.get_all_car_urls()

    # In kết quả để kiểm tra
    if car_brands:
        print("\nDanh sách thương hiệu xe tìm được:")
        for item in car_brands:
            print(f"- {item['brand']}: {item['url']}")
    else:
        print("Không tìm thấy thương hiệu nào.")

Đã truy cập trang web thành công.
Đã tìm thấy: Financing - https://www.cars.com/new-cars/finance/
Đã tìm thấy: Find a Car - https://www.cars.com/new-cars/shopping/
Đã tìm thấy: Find a Dealer - https://www.cars.com/new-cars/dealers/buy/
Đã tìm thấy: Listings by City - https://www.cars.com/new-cars/sitemap/city-listings/
Đã tìm thấy: Certified Pre-Owned - https://www.cars.com/new-cars/cpo/
Đã tìm thấy: Car Payment Calculators - https://www.cars.com/new-cars/car-loan-calculator/
Đã tìm thấy: Car Reviews & Ratings - https://www.cars.com/new-cars/reviews/
Đã tìm thấy: Compare Side by Side - https://www.cars.com/new-cars/research/compare/
Đã tìm thấy: Fraud Awareness - https://www.cars.com/new-cars/fraud-awareness/
Đã tìm thấy: Sell Your Car - https://www.cars.com/new-cars/sell/
Lỗi khi crawl: [Errno 2] No such file or directory: 'dataset/car_brands.json'

Danh sách thương hiệu xe tìm được:
- Financing: https://www.cars.com/new-cars/finance/
- Find a Car: https://www.cars.com/new-cars/shoppi

In [7]:
import requests
from bs4 import BeautifulSoup
import json
import os

def create_session():
    """Tạo session requests với headers cố định"""
    session = requests.Session()
    session.headers.update({
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
    })
    return session

def get_soup(session, url):
    """Lấy BeautifulSoup từ URL"""
    try:
        response = session.get(url)
        response.raise_for_status()
        return BeautifulSoup(response.content, 'html.parser')
    except requests.RequestException as e:
        print(f"Lỗi khi truy cập {url}: {e}")
        return None

def save_to_json(filename, data):
    """Lưu dữ liệu vào file JSON"""
    # Đọc dữ liệu hiện tại (nếu có)
    try:
        if os.path.exists(filename):
            with open(filename, 'r', encoding='utf-8') as f:
                existing_data = json.load(f)
        else:
            existing_data = []
    except (json.JSONDecodeError, FileNotFoundError):
        existing_data = []
    
    # Mở rộng dữ liệu
    existing_data.extend(data)
    
    # Tạo thư mục nếu chưa tồn tại
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    
    # Lưu dữ liệu
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(existing_data, f, ensure_ascii=False, indent=2)

def get_all_car_urls(base_url):
    """Lấy danh sách các URL thương hiệu xe"""
    all_car_urls = []
    
    # Tạo session
    session = create_session()
    
    try:
        # Lấy HTML từ base_url
        soup = get_soup(session, base_url)
        
        if not soup:
            print("Không thể lấy dữ liệu từ trang web.")
            return all_car_urls
        
        # Tìm danh sách thương hiệu xe
        brand_section = soup.find("ul", class_="sds-list")
        
        if not brand_section:
            print("Không tìm thấy danh sách thương hiệu.")
            return all_car_urls
        
        # Lấy tất cả các liên kết thương hiệu
        brand_links = brand_section.find_all("a")
        
        for link in brand_links:
            brand_name = link.text.strip()
            brand_href = link.get("href")
            
            if brand_href:
                # Xử lý URL tuyệt đối hoặc tương đối
                full_url = brand_href if brand_href.startswith("http") else f"{base_url.rstrip('/')}/{brand_href.lstrip('/')}"
                
                # Thêm vào danh sách
                car_info = {"brand": brand_name, "url": full_url}
                all_car_urls.append(car_info)
                print(f"Đã tìm thấy: {brand_name} - {full_url}")
        
        # Lưu kết quả vào file JSON
        if all_car_urls:
            save_to_json("dataset/car_brands.json", all_car_urls)
            print("Đã lưu danh sách thương hiệu vào 'dataset/car_brands.json'.")
    
    except Exception as e:
        print(f"Lỗi khi crawl: {e}")
    
    return all_car_urls

def main():
    # URL để crawl
    base_url = "https://www.cars.com/new-cars/"
    
    # Lấy danh sách các URL thương hiệu xe
    car_brands = get_all_car_urls(base_url)
    
    # In kết quả
    if car_brands:
        print("\nDanh sách thương hiệu xe tìm được:")
        for item in car_brands:
            print(f"- {item['brand']}: {item['url']}")
    else:
        print("Không tìm thấy thương hiệu nào.")

# Chạy chính
if __name__ == "__main__":
    main()

Đã tìm thấy: Financing - https://www.cars.com/new-cars/finance/
Đã tìm thấy: Find a Car - https://www.cars.com/new-cars/shopping/
Đã tìm thấy: Find a Dealer - https://www.cars.com/new-cars/dealers/buy/
Đã tìm thấy: Listings by City - https://www.cars.com/new-cars/sitemap/city-listings/
Đã tìm thấy: Certified Pre-Owned - https://www.cars.com/new-cars/cpo/
Đã tìm thấy: Car Payment Calculators - https://www.cars.com/new-cars/car-loan-calculator/
Đã tìm thấy: Car Reviews & Ratings - https://www.cars.com/new-cars/reviews/
Đã tìm thấy: Compare Side by Side - https://www.cars.com/new-cars/research/compare/
Đã tìm thấy: Fraud Awareness - https://www.cars.com/new-cars/fraud-awareness/
Đã tìm thấy: Sell Your Car - https://www.cars.com/new-cars/sell/
Đã lưu danh sách thương hiệu vào 'dataset/car_brands.json'.

Danh sách thương hiệu xe tìm được:
- Financing: https://www.cars.com/new-cars/finance/
- Find a Car: https://www.cars.com/new-cars/shopping/
- Find a Dealer: https://www.cars.com/new-cars/d